In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from sklearn.utils import shuffle
from tensorflow.keras import optimizers

import numpy as np

from datetime import datetime

In [2]:
BATCH = 128
datagen = ImageDataGenerator(rescale=1./255)
    
train_data = datagen.flow_from_directory('/kaggle/input/intel-image-classification/seg_train/seg_train/',
                                        target_size=(150, 150),
                                        batch_size=14034,
                                        class_mode='categorical',
                                        shuffle=True)

test_data = datagen.flow_from_directory('/kaggle/input/intel-image-classification/seg_test/seg_test/',
                                        target_size=(150, 150),
                                        batch_size=3000,
                                        class_mode='categorical',
                                        shuffle=True)


#train_images, train_labels = train_data.next()
#test_images, test_labels = test_data.next()

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [3]:
train_images, train_labels = train_data.next()
test_images, test_labels = test_data.next()

In [4]:
train_images.shape

(14034, 150, 150, 3)

In [5]:
#train_images = np.reshape( train_images,  (len(train_images),  150 ,  150 ,  3 ))   # adapt this if using `channels_first` image data format
#test_images =  np.reshape( test_images,  (len(test_images),  150 ,  150 ,  3 ))

### Сеть 1

Простой автокодировщик

In [6]:
from keras.layers import Input, Dense, Flatten, Reshape
from keras.models import Model
def create_dense_ae():
    
    encoding_dim = 1000

    input_img = Input(shape=(150, 150, 3))
    flat_img = Flatten()(input_img)

    encoded = Dense(encoding_dim, activation='relu')(flat_img)
    
    input_encoded = Input(shape=(encoding_dim,))
    flat_decoded = Dense(150*150*3, activation='relu')(input_encoded)
    decoded = Reshape((150, 150, 3))(flat_decoded)

    encoder = Model(input_img, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_img, decoder(encoder(input_img)), name="autoencoder")
    return encoder, decoder, autoencoder

Using TensorFlow backend.


In [7]:
encoder, decoder, autoencoder = create_dense_ae()
autoencoder.summary()

autoencoder.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

time_start = datetime.now()
autoencoder.fit( train_images, train_images, epochs= 20, batch_size= 128, shuffle= True, 
                validation_data= (test_images,  test_images))
print('Time: ', datetime.now() - time_start)

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
encoder (Model)              (None, 1000)              67501000  
_________________________________________________________________
decoder (Model)              (None, 150, 150, 3)       67567500  
Total params: 135,068,500
Trainable params: 135,068,500
Non-trainable params: 0
_________________________________________________________________
Train on 14034 samples, validate on 3000 samples
Epoch 1/20
14034/14034 [==============================] - 18s 1ms/step - loss: 7.0550 - accuracy: 0.3519 - val_loss: 6.7996 - val_accuracy: 0.3858
Epoch 2/20
14034/14034 [==============================] - 17s 1ms/step - loss: 5.3675 - accuracy: 0.3946 - val_loss: 4.3050 - val_accuracy: 0.4275
Epoch 3/20
14034/14034 [====

In [8]:
autoencoder.layers[1].summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 67500)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              67501000  
Total params: 67,501,000
Trainable params: 67,501,000
Non-trainable params: 0
_________________________________________________________________


In [9]:
def dense_auto(autoencoder):
    input_img = Input(shape=(150, 150, 3))    
    encoder_dim = Input(shape=(1000,))
    
    dens_1 = Dense(512, activation='relu')(encoder_dim)
    dens_2 = Dense(256, activation='relu')(dens_1)
    dens_3 = Dense(128, activation='relu')(dens_2)
    dens_4 = Dense(64, activation='relu')(dens_3)
    dens_5 = Dense(32, activation='relu')(dens_4)
    dens_6 = Dense(16, activation='relu')(dens_5)
    dens = Dense(6, activation='softmax')(dens_6)

    
    auto = Model(input_img, autoencoder.layers[1](input_img), name="encoder")    
    dens_ = Model(encoder_dim, dens, name="dens_")
    fc = Model(input_img, dens_(auto(input_img)), name="fc")
    
    fc.summary()
    fc.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    time_start = datetime.now()
    fc.fit(train_images, train_labels, epochs=20, batch_size=128, shuffle=True, validation_data=(test_images,  test_labels))
    print('Time: ', datetime.now() - time_start)
    
    return fc

dense_auto(autoencoder)

Model: "fc"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
encoder (Model)              (None, 1000)              67501000  
_________________________________________________________________
dens_ (Model)                (None, 6)                 687702    
Total params: 68,188,702
Trainable params: 68,188,702
Non-trainable params: 0
_________________________________________________________________
Train on 14034 samples, validate on 3000 samples
Epoch 1/20
14034/14034 [==============================] - 8s 597us/step - loss: 9.9767 - accuracy: 0.2338 - val_loss: 3.5786 - val_accuracy: 0.2297
Epoch 2/20
14034/14034 [==============================] - 8s 579us/step - loss: 1.6494 - accuracy: 0.4027 - val_loss: 1.4695 - val_accuracy: 0.4330
Epoch 3/20
14034/14034 [=============

Без автокодировщика

In [10]:
def dense():
    input_img = Input(shape=(150, 150, 3))

    flat_img = Flatten()(input_img)
    dens_0 = Dense(1000, activation='relu')(flat_img)
    dens_1 = Dense(512, activation='relu')(dens_0)
    dens_2 = Dense(256, activation='relu')(dens_1)
    dens_3 = Dense(128, activation='relu')(dens_2)
    dens_4 = Dense(64, activation='relu')(dens_3)
    dens_5 = Dense(32, activation='relu')(dens_4)
    dens_6 = Dense(16, activation='relu')(dens_5)
    dens = Dense(6, activation='softmax')(dens_6)
  
    
    fc = Model(input_img, dens, name="fc")
    
    fc.summary()
    fc.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    time_start = datetime.now()   
    fc.fit(train_images, train_labels, epochs=20, batch_size=128, shuffle=True, validation_data=(test_images,  test_labels))
    print('Time: ', datetime.now() - time_start)
    return fc

dense()

Model: "fc"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 67500)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 1000)              67501000  
_________________________________________________________________
dense_11 (Dense)             (None, 512)               512512    
_________________________________________________________________
dense_12 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_13 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_14 (Dense)             (None, 64)                8256     

### Сеть 2

Глубокий автокодировщик

In [11]:
def deep_au():
    input_img = Input(shape=(150, 150, 3))
    flat_img = Flatten()(input_img)

    encoded = Dense(1024, activation='relu')(flat_img)
    encoded = Dense(512, activation='relu')(encoded)
    encoded = Dense(256, activation='relu')(encoded)
    input_encoded = Input(shape=(256,))
##########################################################################
    decoded = Dense(512, activation='relu')(input_encoded)
    decoded = Dense(1024, activation='relu')(decoded)
    flat_decoded = Dense(150*150*3, activation='relu')(decoded)
    decoded = Reshape((150, 150, 3))(flat_decoded)
    
    encoder = Model(input_img, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_img, decoder(encoder(input_img)), name="autoencoder")

    
    autoencoder.summary()
    
    
    autoencoder.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    time_start = datetime.now()    
    autoencoder.fit(train_images, train_images,
                epochs=20,
                batch_size=128,
                shuffle=True,
                validation_data=(test_images, test_images))
    print('Time: ', datetime.now() - time_start)
    return autoencoder

autoencoder2 = deep_au()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
encoder (Model)              (None, 256)               69777152  
_________________________________________________________________
decoder (Model)              (None, 150, 150, 3)       69844396  
Total params: 139,621,548
Trainable params: 139,621,548
Non-trainable params: 0
_________________________________________________________________
Train on 14034 samples, validate on 3000 samples
Epoch 1/20
14034/14034 [==============================] - 17s 1ms/step - loss: 7.2509 - accuracy: 0.3412 - val_loss: 6.8879 - val_accuracy: 0.3339
Epoch 2/20
14034/14034 [==============================] - 17s 1ms/step - loss: 6.7764 - accuracy: 0.3424 - val_loss: 6.6662 - val_accuracy: 0.3822
Epoch 3/20
14034/14034 [====

In [12]:
autoencoder2.layers[1].summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 67500)             0         
_________________________________________________________________
dense_18 (Dense)             (None, 1024)              69121024  
_________________________________________________________________
dense_19 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_20 (Dense)             (None, 256)               131328    
Total params: 69,777,152
Trainable params: 69,777,152
Non-trainable params: 0
_________________________________________________________________


In [13]:
def dense2_auto(autoencoder2):
    input_img = Input(shape=(150, 150, 3))    
    encoder_dim = Input(shape=(256,))
    
    dens_1 = Dense(128, activation='relu')(encoder_dim)
    dens_2 = Dense(64, activation='relu')(dens_1)
    dens_3 = Dense(32, activation='relu')(dens_2)
    dens_4 = Dense(16, activation='relu')(dens_3)

    dens = Dense(6, activation='softmax')(dens_4)

    
    auto = Model(input_img, autoencoder2.layers[1](input_img), name="encoder")    
    dens_ = Model(encoder_dim, dens, name="dens_")
  
    fc = Model(input_img, dens_(auto(input_img)), name="fc")
    
    fc.summary()
    fc.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    time_start = datetime.now()
    fc.fit(train_images, train_labels, epochs=20, batch_size=128, shuffle=True, validation_data=(test_images,  test_labels))
    print('Time: ', datetime.now() - time_start)
    return fc

dense2_auto(autoencoder2)

Model: "fc"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
encoder (Model)              (None, 256)               69777152  
_________________________________________________________________
dens_ (Model)                (None, 6)                 43862     
Total params: 69,821,014
Trainable params: 69,821,014
Non-trainable params: 0
_________________________________________________________________
Train on 14034 samples, validate on 3000 samples
Epoch 1/20
14034/14034 [==============================] - 8s 595us/step - loss: 25.3691 - accuracy: 0.1827 - val_loss: 2.0544 - val_accuracy: 0.1370
Epoch 2/20
14034/14034 [==============================] - 8s 576us/step - loss: 1.8381 - accuracy: 0.2324 - val_loss: 1.6935 - val_accuracy: 0.2960
Epoch 3/20
14034/14034 [============

Без автокодировщика

In [14]:
def dense2():
    input_img = Input(shape=(150, 150, 3))    
    flat_img = Flatten()(input_img)
    
    dens_1 = Dense(1024, activation='relu')(flat_img)
    dens_2 = Dense(512, activation='relu')(dens_1)
    dens_3 = Dense(256, activation='relu')(dens_2)
    dens_4 = Dense(128, activation='relu')(dens_3)
    dens_5 = Dense(64, activation='relu')(dens_4)
    dens_6 = Dense(32, activation='relu')(dens_5)
    dens_7 = Dense(16, activation='relu')(dens_6)

    dens = Dense(6, activation='softmax')(dens_7)
  
    fc = Model(input_img, dens, name="fc")
    
    fc.summary()
    fc.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    time_start = datetime.now()
    fc.fit(train_images, train_labels, epochs=20, batch_size=128, shuffle=True, validation_data=(test_images,  test_labels))
    print('Time: ', datetime.now() - time_start)
    return fc

dense2()

Model: "fc"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 150, 150, 3)       0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 67500)             0         
_________________________________________________________________
dense_29 (Dense)             (None, 1024)              69121024  
_________________________________________________________________
dense_30 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_31 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_32 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_33 (Dense)             (None, 64)                8256     

### Сеть 3

Сверточный автокодировщик

In [15]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
def conv_ae():    
    input_img = Input(shape=(150, 150, 3))
    #flat_img = Flatten()(input_img)
    
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    print(encoded.shape)
    input_encoded = Input(shape=(19, 19, 8))
##############################################################
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(input_encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='relu')(x)
    print(decoded.shape)
    
    encoder = Model(input_img, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_img, decoder(encoder(input_img)), name="autoencoder")
    
    autoencoder.summary()
    
    
    autoencoder.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    time_start = datetime.now()
    autoencoder.fit(train_images, train_images,
                epochs=20,
                batch_size=128,
                shuffle=True,
                validation_data=(test_images, test_images))
    print('Time: ', datetime.now() - time_start)
    return autoencoder

autoencoder3 = conv_ae()

(None, 19, 19, 8)
(None, 150, 150, 3)
Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 150, 150, 3)       0         
_________________________________________________________________
encoder (Model)              (None, 19, 19, 8)         6680      
_________________________________________________________________
decoder (Model)              (None, 150, 150, 3)       7259      
Total params: 13,939
Trainable params: 13,939
Non-trainable params: 0
_________________________________________________________________
Train on 14034 samples, validate on 3000 samples
Epoch 1/20
14034/14034 [==============================] - 24s 2ms/step - loss: 1.5915 - accuracy: 0.4550 - val_loss: 1.4683 - val_accuracy: 0.4178
Epoch 2/20
14034/14034 [==============================] - 20s 1ms/step - loss: 1.4589 - accuracy: 0.5702 - val_loss: 1.4646 - val_accuracy: 0.4438


In [16]:
autoencoder3.layers[1].summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 150, 150, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 16)        4624      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 38, 38, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 38, 38, 8)         1160      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 19, 19, 8)         0   

In [17]:
def dense3_auto(autoencoder3):
    input_img = Input(shape=(150, 150, 3))    
    
    encoder_dim = Input(shape=(19, 19, 8))
    flat_encoder_dim = Flatten()(encoder_dim)
    
    dens_1 = Dense(1024, activation='relu')(flat_encoder_dim)
    dens_2 = Dense(512, activation='relu')(dens_1)
    dens_3 = Dense(256, activation='relu')(dens_2)
    dens_4 = Dense(128, activation='relu')(dens_3)
    dens_5 = Dense(64, activation='relu')(dens_4)
    dens_6 = Dense(32, activation='relu')(dens_5)
    dens_7 = Dense(16, activation='relu')(dens_6)

    dens = Dense(6, activation='softmax')(dens_7)

    auto = Model(input_img, autoencoder3.layers[1](input_img), name="encoder")    
    dens_ = Model(encoder_dim, dens, name="dens_")
    
    fc = Model(input_img, dens_(auto(input_img)), name="fc")
    
    fc.summary()
    fc.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    time_start = datetime.now()
    fc.fit(train_images, train_labels, epochs=20, batch_size=128, shuffle=True, validation_data=(test_images,  test_labels))
    print('Time: ', datetime.now() - time_start)
    return fc

dense3_auto(autoencoder3)

Model: "fc"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 150, 150, 3)       0         
_________________________________________________________________
encoder (Model)              (None, 19, 19, 8)         6680      
_________________________________________________________________
dens_ (Model)                (None, 6)                 3658326   
Total params: 3,665,006
Trainable params: 3,665,006
Non-trainable params: 0
_________________________________________________________________
Train on 14034 samples, validate on 3000 samples
Epoch 1/20
14034/14034 [==============================] - 9s 665us/step - loss: 1.3889 - accuracy: 0.4440 - val_loss: 1.0791 - val_accuracy: 0.5780
Epoch 2/20
14034/14034 [==============================] - 9s 636us/step - loss: 1.0220 - accuracy: 0.6109 - val_loss: 1.0531 - val_accuracy: 0.6057
Epoch 3/20
14034/14034 [===============

Без автокодировщика

In [18]:
def dense3():
    input_img = Input(shape=(150, 150, 3))    
  
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    print(x.shape)

    x = Flatten()(x)
    print(x.shape)
    dens_1 = Dense(1024, activation='relu')(x)
    dens_2 = Dense(512, activation='relu')(dens_1)
    dens_3 = Dense(256, activation='relu')(dens_2)
    dens_4 = Dense(128, activation='relu')(dens_3)
    dens_5 = Dense(64, activation='relu')(dens_4)
    dens_6 = Dense(32, activation='relu')(dens_5)
    dens_7 = Dense(16, activation='relu')(dens_6)

    dens = Dense(6, activation='softmax')(dens_7)

    
    fc = Model(input_img, dens, name="fc")
    
    fc.summary()
    fc.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    time_start = datetime.now()
    fc.fit(train_images, train_labels, epochs=20, batch_size=128, shuffle=True, validation_data=(test_images,  test_labels))
    print('Time: ', datetime.now() - time_start)
    return fc

dense3()

(None, 19, 19, 8)
(None, None)
Model: "fc"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 150, 150, 3)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 75, 75, 16)        4624      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 38, 38, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 38, 38, 8)         1160      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (Non